In [ ]:
## We Analyze Scans of Arteries and Tumors
## We aim to classify, if there exists a 2d slice of the 3d scan where the tumor encloses the arterie completely
## The scans contain one tumor and two arteries


In [75]:
import os
import numpy as np
import queue
import time
# uncomment the next line if you haven't worked with nibabel yet
#!pip install nibabel
from nibabel.testing import data_path
import nibabel as nib


In [115]:
## Loading Data: 
## 0 = neither, 1 = Tumor, 2 = right arterie, 3 = left arterie
data = nib.load('009_pre_seg.nii').get_fdata()

In [116]:
## Tuple add, this should exist somewhere as a lib function, but ill write it anyway
def my_add(xs, ys):
    return (xs[0] + ys[0],xs[1]+ys[1])

def good_tuple(xs):
    x = xs[0]
    y = xs[1]
    if x >= 0 and x < 256 and y >= 0 and y < 256:
        return True
    return False

In [117]:
## This function outputs true if the tumor encloses the artery completely on this slice!
neighbours = [(1,0),(0,1),(-1,0),(0,-1),(1,1),(1,-1),(-1,-1),(-1,1)]
def check_Tumor(arterie_num, slice):
    Flag = False ## Check Flag if Tumor exists on this slice
    ##slice[0,0] = arterie_num
    q = queue.Queue() ## create queue for bfs
    ## put every artery bit onto the queue!
    for i in range(0,slice.shape[0]):
        for j in range(0,slice.shape[1]):
            if slice[i,j] == arterie_num:
                q.put((i,j))
            if slice[i,j] == 1:
                Flag = True
    ## If Flag = False here, then there is no tumor on this slice therefore it can't enclose the artery!
    if not Flag:
        return (False,False)
    ## Do BFS to "grow" artery
    while q.qsize() > 0:
        current_idx = q.get()
        ## now check all neighbours
        for nex in neighbours:
            neigh = my_add(current_idx,nex)
            if good_tuple(neigh) and slice[neigh] == 0:
                slice[neigh] = arterie_num
                q.put(neigh)
    ## We now check whether there exists a way for the grown artery to touch the border
    ## If that were the case, then the virus does not enclose the artery on that slice
    N = 256
    for idx in range(0,N):
        if slice[0,idx] == arterie_num or slice[N-1,idx] == arterie_num or slice[idx,0] == arterie_num or slice[idx,N-1] == arterie_num:
                return (False,Flag)
    return (True,Flag)
        
    

In [118]:
start = time.time()
slice_num = 0

for cur_slice in data: ## I assume this loops through the z axis, not sure tho!
    enclosed_right = check_Tumor(2, cur_slice.copy()) ## check right arterie
    enclosed_left = check_Tumor(3, cur_slice.copy()) ## check left arterie
    if enclosed_right[0] and enclosed_right[1]:
        print("on Slice: " + str(slice_num) + " the tumor encloses the right artery completely!")
    if enclosed_left[0] and enclosed_left[1]:
        print("on Slice: " + str(slice_num) + " the tumor encloses the left artery completely!")
    slice_num = slice_num + 1
end = time.time()
print("This procedure took: " + str(end-start))

on Slice: 114 the tumor encloses the right artery completely!
on Slice: 115 the tumor encloses the right artery completely!
on Slice: 116 the tumor encloses the right artery completely!
on Slice: 117 the tumor encloses the right artery completely!
on Slice: 118 the tumor encloses the right artery completely!
on Slice: 119 the tumor encloses the right artery completely!
on Slice: 119 the tumor encloses the left artery completely!
on Slice: 120 the tumor encloses the right artery completely!
on Slice: 120 the tumor encloses the left artery completely!
on Slice: 121 the tumor encloses the right artery completely!
on Slice: 121 the tumor encloses the left artery completely!
on Slice: 122 the tumor encloses the right artery completely!
on Slice: 122 the tumor encloses the left artery completely!
on Slice: 123 the tumor encloses the right artery completely!
on Slice: 123 the tumor encloses the left artery completely!
on Slice: 124 the tumor encloses the right artery completely!
on Slice: 124